<a href="https://colab.research.google.com/github/Potdooshami/The-4th-POSTECH-Youth-Mathematical-Artificial-Intelligence-Academy-Public/blob/main/PYMAIA4_day1_%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80%2C%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



[![](https://www.gstatic.com/images/branding/product/2x/sheets_2020q4_48dp.png)데이터 수집용](https://docs.google.com/spreadsheets/d/1YlJ18eApWzd3qbpds8q4-yrFn5i6GgGfiDymwUR8734/edit?gid=0#gid=0)  [![](https://www.gstatic.com/images/branding/product/2x/slides_2020q4_48dp.png)분석 결과 공유](https://docs.google.com/presentation/d/1WZ-l75mrvAapqYKdPfsYUPnnDzMqVdsJxnmaGf4fLPY/edit?usp=sharing)


#1 데이터 전처리 및 탐색


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import io
import pandas as pd


## 데이터 수집

In [ ]:
df = pd.read_csv(io.StringIO('''
ID,키,발 사이즈,"성별(남,여)"
차은우,167,250,여
홍지후,162,245,여
"ㅡ""ㅡ",164,255,여
  ll,168,240,여
장윤아,162,240,여
수혀니,165,230,여
미미민지,158,230,여
김노겸,159,240,여
애기공주,198,320,여
집가고싶다고,167,230,여
ㅇ,186,280,남
ㅇ[]ㅇ,161,250,여
송재민,184,290,남
정세헌,174,270,남
권하율,170,270,남
박지원,181,300,남
류민석,151,220,여
악,156,245,여
이세한,176,265,남
ㅇㅅㅇ,162,245,남
이연지,165,245,여
어니ㅏㅇ머,162,250,여
윤하준,175,265,상남자
이름,176,270,남
'''))
df

## 데이터 전처리

한국어 -> 영어

In [ ]:
df.columns = ['ID','height','shoe size','sex']

결측치 처리

In [ ]:
gender_map = {
    '남': 'male',
    '상남자': 'male',  # '상남자'도 'male'로 매핑
    '여': 'Female'
}
df['sex'] = df['sex'].map(gender_map)
df

In [ ]:
df = df.iloc[:,1:]

불필요한 데이터 제거

In [ ]:
df

## 데이터 시각화

시각화1: 산점도 플롯

In [ ]:
xs = df['height'].values
ys = df['shoe size'].values
fig,ax = plt.subplots()
ax.plot(xs,ys,'o')
plt.ylabel('Shoe Size')
plt.xlabel('Height')


시각화2 = 시각화1 + 성별 정보  
->좋은 시각화는 데이터를 더 명료하게 보여준다.

In [ ]:
import seaborn as sns
sns.scatterplot(data=df, x='height', y='shoe size',hue='sex')

이상치 제거

In [ ]:
is_outlier = df['height'].values>190
df = df[~is_outlier]
sns.scatterplot(data=df, x='height', y='shoe size',hue='sex')

# 2 선형 회귀

## 키로 발사이즈 예측하기

In [ ]:
xs = df['height'].values
ys = df['shoe size'].values
fig,ax = plt.subplots()
ax.plot(xs,ys,'o')
plt.ylabel('Shoe Size')
plt.xlabel('Height')

##파라미터 튜닝: m,c를 조절해서 적당한 직선 찾기

In [ ]:
m = 2 # @param {type:"raw"}
c = -80 # @param {type:"raw"}

fig,ax = plt.subplots()
ax.plot(xs,ys,'o',label='Actual Data')
plt.ylabel('Shoe Size')
plt.xlabel('Height')
my_figure = fig


xx = np.arange(150,200)
plt.plot(xx,m*xx+c,label='Predict')
plt.legend()
plt.title('Linear Regression')

다른 친구들은 어떻게 했을까?



##비용함수: 좋은 직선을 수학적으로 정의하기

In [ ]:
m = 1# @param {type:"number"}
c = 80# @param {type:"number"}
fcn_estimate = lambda x: m*x + c
xs = df['height'].values
ys = df['shoe size'].values
def sort_by_x(x_unsorted, y_unsorted):
    """
    Sorts two lists/arrays based on the values in the first list/array.

    Args:
        x_unsorted: The list or array to sort by.
        y_unsorted: The list or array to sort according to the order of x_unsorted.

    Returns:
        A tuple containing the sorted x_unsorted and y_unsorted.
    """
    # Combine x and y into pairs, sort by x, then separate
    sorted_pairs = sorted(zip(x_unsorted, y_unsorted))
    x_sorted, y_sorted = zip(*sorted_pairs)
    return np.array(x_sorted), np.array(y_sorted)

# Example usage (optional - can be removed or commented out)
# x_unsorted = [5, 2, 8, 1, 9]
# y_unsorted = [10, 4, 16, 2, 18]
# x_sorted, y_sorted = sort_by_x(x_unsorted, y_unsorted)
# print("Sorted x:", x_sorted)
# print("Sorted y:", y_sorted)
def visualize_MSE(fcn_estimate,xs,ys,l_compare=100):
  l = l_compare
  xs,ys = sort_by_x(xs,ys)
  fcn = fcn_estimate
  fig, axs = plt.subplots(1, 2, layout='constrained',figsize=(10, 5))
  plt.sca(axs[0])
  plt.plot(xs,ys,'o',label='Actual Data')
  x_smooth = np.linspace(xs.min(), xs.max(), 100)
  plt.plot(x_smooth,fcn(x_smooth),label='Predict')
  plt.ylabel('Shoe Size')
  plt.xlabel('Height')
  xs = np.array(xs)
  ys = np.array(ys)
  errors = ys - fcn(xs)
  clr_error = []
  for i, error in enumerate(errors):
      if error >= 0:
          clr = 'red'
      else:
          clr = 'blue'
      plt.plot([xs[i], xs[i]], [fcn(xs[i]), ys[i]], color=clr, linestyle='-')
      clr_error.append(clr)
  plt.legend()
  plt.title('Visualize error')





  plt.sca(axs[1])
  # 정사각형 넓이 리스트
  square_areas = errors*errors

  # 정사각형 색상 리스트
  colors = ['red', 'blue', 'green', 'yellow', 'purple']

  # 넓이 합 계산
  total_area = sum(square_areas)

  # 시각화 시작
  ax = axs[1]

  # 정사각형 그리기
  start_x = 0

  for i, area in enumerate(square_areas):
    #area = abs(errors[i])
      side = np.sqrt(area)  # 정사각형 한 변의 길이 계산
      rect = plt.Rectangle((start_x, 0), side, side, facecolor=clr_error[i])
      ax.add_patch(rect)
      start_x += side  # 다음 정사각형 시작 위치 설정

  # 텍스트 추가 (넓이 합 표시)
  #plt.text(0.5, -0.1, f'Total Area: {total_area}',
  #         horizontalalignment='center', verticalalignment='center',
  #         transform=ax.transAxes, fontsize=12)

  # 그래프 설정


  #ax.set_xlim([0, 100])
  #ax.set_ylim([0, 100])  # y축 범위 설정
  ax.set_aspect('equal')  # 정사각형 모양 유지
  plt.title('Sum of Areas of Squares')
  plt.xlabel('Width')
  plt.ylabel('Height')
  plt.plot([0, l, l, 0, 0],[0, 0, l, l, 0],'--',label=f'RMSE={l}')
  lsum = np.sqrt(square_areas.sum())
  xs_sum = np.array([0,1,1,0,0])
  ys_sum = np.array([0,0,1,1,0])
  plt.plot(xs_sum*lsum,ys_sum*lsum,label=f"RMSE(Current Model)={lsum:.3f}")
  plt.legend()

visualize_MSE(fcn_estimate,xs,ys)


x와 y값을 넣어주면 자동으로 m과 c를 정해줄 수는 없을까?

# 3 로지스틱 회귀

## 시그모이드랑 친해지기

[$\sigma(x) = \frac{1}{1 + e^{-(Wx + b)}}$](https://www.desmos.com/calculator/zxl48byttf)



## 발사이즈로 성별 예측하기

In [ ]:
df['is_male'] = (df['sex'].values == 'male')

In [ ]:
sns.scatterplot(data=df, x='shoe size', y='is_male',alpha=.3)

##파라미터 튜닝

In [ ]:
W=1# @param {type:"number"}
b=-250# @param {type:"number"}
xs = df['shoe size'].values
ys = df['is_male'].values
def sigmoid(x, W, b):
    return 1 / (1 + np.exp(-(W * x + b)))
visualize_MSE(lambda x:sigmoid(x, W, b),xs,ys,l_compare=1)


# ...

우리가 오늘 손으로 최적의 값을 컴퓨터에게 시킬 순 없을까?